In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

In [12]:
model = models.vgg19(pretrained=True).features

In [13]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.chosen_features = ['0', '5', '10', '19', '28']
        self.model = models.vgg19(pretrained=True).features[:29]

    def forward(self, x):
        features = []
        for layer_num, layer in enumerate(self.model):
            x = layer(x)
            if str(layer_num) in self.chosen_features:
                features.append(x)
        return features

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [15]:
def load_image(image_name):
    image_size = 356  # Define image size here
    loader = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
    ])
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

In [16]:
# Paths to your images
original_img_path = "D:\\Aries open project\\tubingen.png"
style_img_path = "D:\\Aries open project\\mosaic.jpg"

original_img = load_image(original_img_path)
style_img = load_image(style_img_path)

In [17]:
# Initialize VGG and move to device
model = VGG().to(device).eval()

# Initialize generated image as a copy of original image
generated = original_img.clone().requires_grad_(True)

# Hyperparameters
total_steps = 5000
learning_rate = 0.01  # Adjusted learning rate for faster convergence
alpha = 1  # Weight for content loss
beta = 0.1  # Weight for style loss


In [18]:
optimizer = optim.Adam([generated], lr=learning_rate)


In [19]:
for step in range(total_steps):
    optimizer.zero_grad()

    # Extract features for generated, original, and style images
    generated_features = model(generated)
    original_img_features = model(original_img)
    style_features = model(style_img)

    style_loss = original_loss = 0

    # Compute content loss
    for gen_feature, orig_feature in zip(generated_features, original_img_features):
        original_loss += torch.mean((gen_feature - orig_feature) ** 2)

    # Compute style loss
    for gen_feature, style_feature in zip(generated_features, style_features):
        _, channel, height, width = gen_feature.shape
        G = gen_feature.view(channel, height * width).mm(gen_feature.view(channel, height * width).t())
        A = style_feature.view(channel, height * width).mm(style_feature.view(channel, height * width).t())
        style_loss += torch.mean((G - A) ** 2)

    total_loss = alpha * original_loss + beta * style_loss
    total_loss.backward()
    optimizer.step()

    # Print and save generated image every 200 steps
    if step % 200 == 0:
        print(f"Step [{step}/{total_steps}], Total Loss: {total_loss.item()}")
        save_image(generated, f"generated_{step}.png")


Step [0/5000], Total Loss: 56240972.0
Step [200/5000], Total Loss: 286819.15625
Step [400/5000], Total Loss: 149828.75
Step [600/5000], Total Loss: 111352.6875
Step [800/5000], Total Loss: 92032.7421875
Step [1000/5000], Total Loss: 79605.3828125
Step [1200/5000], Total Loss: 70661.1015625
Step [1400/5000], Total Loss: 63543.35546875
Step [1600/5000], Total Loss: 57543.12890625
Step [1800/5000], Total Loss: 52626.6640625
Step [2000/5000], Total Loss: 48219.48046875
Step [2200/5000], Total Loss: 44090.20703125
Step [2400/5000], Total Loss: 41381.74609375
Step [2600/5000], Total Loss: 281333.96875
Step [2800/5000], Total Loss: 36913.40625
Step [3000/5000], Total Loss: 33296.64453125
Step [3200/5000], Total Loss: 31014.05078125
Step [3400/5000], Total Loss: 30609.748046875
Step [3600/5000], Total Loss: 29326.861328125
Step [3800/5000], Total Loss: 41460.46484375
Step [4000/5000], Total Loss: 29645.767578125
Step [4200/5000], Total Loss: 27236.04296875
Step [4400/5000], Total Loss: 25749.1